In [224]:
import numpy as np
import string

In [225]:
N = 26
n = 10
tickers = np.array(list(string.ascii_uppercase))
years = 20
prices = np.random.random((1,N))*10 + 100
indicators = np.random.random((1,N))*100
volumes = np.random.random((1,N))*1000000 + 10000000

In [226]:
for i in range(12*years-1):
    prices = np.vstack((prices, prices[-1] * (1 + (np.random.random(N)-0.5)/5))) # multiply by 1+[-0.1,0.1]
    indicators = np.vstack((indicators, indicators[-1] + 6*np.random.random(N)-3)) # add [-3, 3] 
    volumes = np.vstack((volumes, volumes[-1] * (1 + (np.random.random(N)-0.5)/5))) # multiply by 1+[-0.1,0.1]

In [227]:
class algo(object):
    def __init__(self, initial_prices, initial_indicators, initial_volume): 
                     # intial_prices, initial_indicators, initial_volume are N-d arrays
        self.months = 0
        self.avail_capital = 100000000
        
        self.price_arr = initial_prices
        self.indicator_arr = initial_indicators
        self.volume_arr = initial_volume
        
        # Initialize portfolio
        self.top_indices = np.argsort(-self.indicator_arr)[:n] # argsort is ascending so negate indicator_arr
        
        self.entry_months = np.array([-1]*N) # -1 indicates there is no position so no entry month
        self.entry_months[self.top_indices] = 0
        
        self.shares = np.array([0]*N)
        self.shares[self.top_indices] = (self.avail_capital/n/self.price_arr[self.top_indices]).astype(int)
        self.avail_capital = self.avail_capital - np.sum(self.price_arr*self.shares)        
        
    def process_new_data(self, new_prices, new_indicators, new_volumes):
        self.months += 1
        self.price_arr = np.vstack((self.price_arr, new_prices))
        self.indicator_arr = np.vstack((self.indicator_arr, new_indicators))
        self.volume_arr = np.vstack((self.volume_arr, new_volumes))
        
        self.update_portfolio()
        
    def update_portfolio(self):
        self.new_top_indices = np.argsort(-self.indicator_arr[-1])[:n]
        sell_indices = []
        buy_indices = []
        for x in self.top_indices:# Try to add new indices not currently in portfolio to buy_indices
            if (self.months-self.entry_months[x] >= 12) and (x not in new_top_indices): # if held for >=1yr and not in new top indices
                sell_indices += x
                
        for y in self.new_top_indices: # Add k best new indices to portfolio, k = number of stocks we will sell
            if y not in self.top_indices:
                buy_indices += y
                if len(buy_indices) == len(sell_indices): # Stop once we have same number of buy as sell
                    break
        
        self.execute_sell(sell_indices)
        self.execute_buy(buy_indices)
    
    def execute_sell(self, sell_indices):
        print('selling', tickers[sell_indices])
        for i in sell_indices:
            sell_gain = self.price_arr[-1, i] *self.shares[i]
            transaction_cost = 0.01 * self.shares[i]
            slip_cost = 0
            if self.shares[i] >= 0.1 * self.volume_arr[-1, i]:
                slip_cost = self.price_arr[-1, i] * 0.01 * self.shares[i]
            
            self.avail_capital += sell_gain - transaction_cost - slip_cost # Update capital
            self.shares[i] = 0 # Reset shares
            self.entry_months[i] = -1 # Reset entry_mont to -1 to denote no position
            
        
    def execute_buy(self, buy_indices):
        print('buying', tickers[buy_indices])
        for i in buy_indices:
            buy_money = self.avail_capital/len(buy_indices) # Allocate avail_capital equally 
            buy_price = self.price_arr[-1, i] + 0.01
            n_shares = int(buy_money/buy_price)
            if n_shares >= 0.1 * self.volume_arr[-1, i]:
                buy_price = self.price_arr[-1, i] + 0.01 + 0.1*self.price_arr[-1, i]
                n_shares = int(buy_money/buy_price)
                
            self.shares[i] = n_shares
            self.entry_months[i] = self.months
            self.avail_capital = self.avail_capital - n_shares*buy_price
        